## NG Solve: NavierStokes anfang


defining the geometry

In [2]:
from ngsolve import *
from ngsolve.webgui import *
from netgen.geom2d import *
from netgen.occ import *

a = 28
b = 4

shape = MoveTo(0,0).Rectangle(a,b).Face()
pillar = Circle((10,2),0.5).Face()

rect = shape - pillar 

rect.edges.name = "shore"      # TODO declaring SOURCE and SINK is still not done
pillar.edges.name = "wall"

shape.edges.Min(X).name = "source"
shape.edges.Max(X).name = "sink"


geo = OCCGeometry(rect,dim=2)

#mesh = Mesh(geo.GenerateMesh(maxh=0.2)).Curve(3)
# mesh.GetBoundaries()

creating a mesh

In [3]:
#from ngsolve import *
#from ngsolve.webgui import Draw
mesh = Mesh(geo.GenerateMesh(maxh=2))
mesh.Curve(3)
Draw(mesh)

#Draw(b,mesh,"Grundhöhe")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

Vectorial approximation space

In [ ]:
#TODO

#order = 2
#fes = L2(mesh,order=order)**3

fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space


312

initial & boundary conditions

In [5]:

u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

#short version:
#u,v = fes.TnT()         # U is solution-function, V is test-function


define the solution of poisson equation: f!!!

we have a pde-problem - here its a poisson equation with
- laplace(u) = f_pde              on Omega
- u = 0                           on boundary of Omega

In [6]:
f_pde = 5*x*sin(100*x*y)

assemble bilinear/linear forms

In [7]:

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx    # here was the lemma of green used, grad(u)*grad(v) = laplace(u) * v 
a.Assemble()

f = LinearForm(fes)
f += f_pde*v*dx
f.Assemble()

#short version:
#a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
#f = LinearForm(x*v*dx).Assemble()


solve the system

In [8]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…